In [1]:
import pickle
import numpy as np
from metrics import mapk
from collections import defaultdict

In [19]:
# подразумеваются результаты для 2к определенных пользователей, иначе надо добавить проверку по id
class Ensemble():
    def __init__(self, most_popular_prediction, item_based_prediction, user_based_prediction, actual_purchases):
        '''
        :param most_popular_predict: dictionary like {user_id: [predicted products]}
        :param item_based_predict: dictionary like {user_id: [predicted products]}
        :param user_based_predict: dictionary like {user_id: [predicted products]}
        :param actual_purchases: dictionary like {user_id: [actual products]}
        '''
        self.predictions = {'most popular': most_popular_prediction,
                            'item based'  : item_based_prediction,
                            'user based'  : user_based_prediction}
        self.actual = actual_purchases
        
    # AP@k
    def apk(self, actual, predicted, k=15):
        
        if len(predicted) > k:
            predicted = predicted[:k]

        score = 0.0
        num_hits = 0.0

        for i,p in enumerate(predicted):
            if p in actual and p not in predicted[:i]:
                num_hits += 1.0
                score += num_hits / (i+1.0)

        if not actual:
            return 0.0

        return score / min(len(actual), k)
    
    def fit(self, users_ids=False):
        '''
        :param user_ids: list with IDs of interesting users (2k)
        :return: dict {user_id: [products ids from best prediction]}
        '''
        if not users_ids:
            users_ids = [*self.predictions['user based']]

        self.predictions, self.predictions_models = self.get_best_predictions(users_ids=users_ids)
        return self.predictions, self.predictions_models
    
    def get_best_predictions(self, users_ids):
        predictions = {}
        predictions_models = {}
        self.current_users_actual = {}
        
        d = {0: 'most popular', 1: 'item based', 2:'user based'}
        
        for user_id in users_ids:
            if user_id in self.predictions['most popular'] and \
                user_id in self.predictions['item based'] and \
                user_id in self.predictions['user based'] and \
                user_id in self.actual:
                
                # в actual нет юзера с id7, поэтому проверку оставляем
                
                metrics = np.array([self.apk(self.actual[user_id], self.predictions['most popular'][user_id]),
                                    self.apk(self.actual[user_id], self.predictions['item based'][user_id]),
                                    self.apk(self.actual[user_id], self.predictions['user based'][user_id])])
                
                # результатом будет предсказание и название модели, сделавшей его
                predictions_models[user_id] = d[np.argmax(metrics)]
                predictions[user_id] = self.predictions[d[np.argmax(metrics)]][user_id]
                self.current_users_actual[user_id] = self.actual[user_id]
                
        return predictions, predictions_models
    
    def predict(self, user_id=False):
        if user_id:
            return self.predictions[user_id], self.predictions_models[user_id]
        else:
            return self.predictions, self.predictions_models
        
    def get_actual(self):
        return self.current_users_actual

## Пример работы

### Загрузка необходимых данных

In [3]:
# необходимые данные: 3 словаря с предсказаниями моделей для интересных пользователей, 
#                     список с id интересных пользователей (проверяли все ли на месте),
#                     словарь с актуальными покупками (test)

with open('data/2_iter_user_based_interesting_users.pickle', 'rb') as f:
    user_based_prediction = pickle.load(f)
    
with open('data/2_iter_item_based_interesting_users.pickle', 'rb') as f:
    item_based_prediction = pickle.load(f)
    
with open('data/2_iter_most_popular_full.pickle', 'rb') as f:
    most_popular_prediction = pickle.load(f)

with open('users_subsample.pickle', 'rb') as input:
        users_ids = pickle.load(input)
        
with open('data/2_iter_full_actual_purchases.pickle', 'rb') as f:
    actual_purchases = pickle.load(f)



In [67]:
# число индексов генерящихся рандомно - надо подфиксить, либо пошарить один файл
k = 0
for _id in users_ids:
    if _id not in item_based_prediction:
        k += 1
k

995

In [20]:
# создание, обучение и получение результатов
predictor = Ensemble(most_popular_prediction, item_based_prediction, user_based_prediction, actual_purchases)
predictor.fit(users_ids)
predictions = predictor.predict()

# исходной actual - для всех юзеров, это - укороченное под интересных
current_actual = predictor.get_actual()

In [6]:
# запись в файлы результатов

predictions, predictions_models = predictions

with open('data/2_iter_ensenble_predictions.pickle', 'wb') as f:
    pickle.dump(predictions, f)
    
with open('data/2_iter_ensenble_predictions_models.pickle', 'wb') as f:
    pickle.dump(predictions_models, f)

## Некоторые результаты

#### 1 итерация

In [37]:
# соотношение кто для скольких оказался лучшим
d = defaultdict(int)
for model in predictions[1].values():
    d[model] +=1
d

defaultdict(int, {'item based': 1120, 'user based': 517, 'most popular': 239})

#### 2 итерация

In [9]:
# соотношение кто для скольких оказался лучшим
d = defaultdict(int)
for model in predictions[1].values():
    d[model] +=1
d

defaultdict(int, {'item based': 1169, 'user based': 462, 'most popular': 245})

#### 1 итерация

In [38]:
# метрички для ансамбля 
current_actual = predictor.get_actual()
print(mapk(current_actual.values(), predictions[0].values(), k=1),
      mapk(current_actual.values(), predictions[0].values(), k=5),
      mapk(current_actual.values(), predictions[0].values(), k=10),
      mapk(current_actual.values(), predictions[0].values(), k=30))

0.6556503198294243 0.4102746683250415 0.337125098746361 0.2619891542346653


#### 2 итерация

In [10]:
# метрички для ансамбля 
current_actual = predictor.get_actual()
print(mapk(current_actual.values(), predictions[0].values(), k=1),
      mapk(current_actual.values(), predictions[0].values(), k=5),
      mapk(current_actual.values(), predictions[0].values(), k=10),
      mapk(current_actual.values(), predictions[0].values(), k=30))

0.9792110874200426 0.8950426439232408 0.7744083367177717 0.5236909980078231


#### 1 итерация

In [39]:
# исходные метрички для user_based

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
user_based_pr = [user_based_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), user_based_pr, k=1),
      mapk(current_actual.values(), user_based_pr, k=5),
      mapk(current_actual.values(), user_based_pr, k=10),
      mapk(current_actual.values(), user_based_pr, k=30))

0.5549040511727079 0.2933139362710258 0.22946445317347502 0.19104062794227183


#### 2 итерация

In [12]:
# исходные метрички для user_based

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
user_based_pr = [user_based_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), user_based_pr, k=1),
      mapk(current_actual.values(), user_based_pr, k=5),
      mapk(current_actual.values(), user_based_pr, k=10),
      mapk(current_actual.values(), user_based_pr, k=30))

0.7457356076759062 0.4374413646055437 0.28552936846380345 0.20487215296606043


#### 1 итерация

In [40]:
# исходные метрички для most popular

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
most_popular_pr = [most_popular_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), most_popular_pr, k=1),
      mapk(current_actual.values(), most_popular_pr, k=5),
      mapk(current_actual.values(), most_popular_pr, k=10),
      mapk(current_actual.values(), most_popular_pr, k=30))

0.17643923240938167 0.11234956171523336 0.09158224205341933 0.0654180065089918


#### 2 итерация

In [13]:
# исходные метрички для most popular

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
most_popular_pr = [most_popular_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), most_popular_pr, k=1),
      mapk(current_actual.values(), most_popular_pr, k=5),
      mapk(current_actual.values(), most_popular_pr, k=10),
      mapk(current_actual.values(), most_popular_pr, k=30))

0.3208955223880597 0.2637668798862829 0.2186482976275087 0.14164162404273184


#### 1 итерация

In [41]:
# исходные метрички для item_based

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
item_based_pr = [item_based_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), item_based_pr, k=1),
      mapk(current_actual.values(), item_based_pr, k=5),
      mapk(current_actual.values(), item_based_pr, k=10),
      mapk(current_actual.values(), item_based_pr, k=30))

0.517590618336887 0.3505561478322672 0.30057917284125213 0.23138538421331833


#### 2 итерация

In [14]:
# исходные метрички для item_based

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
item_based_pr = [item_based_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), item_based_pr, k=1),
      mapk(current_actual.values(), item_based_pr, k=5),
      mapk(current_actual.values(), item_based_pr, k=10),
      mapk(current_actual.values(), item_based_pr, k=30))

0.7857142857142857 0.7130063965884861 0.6310394667817376 0.4211517702354683
